# Unweighted Connected Components
_Connected Components_ or _UnionFind_ basically finds sets of connected nodes where each node is reachable from any other node in the same set. 
In graph theory, a connected component of an undirected graph is a subgraph in which any two vertices are connected to each other by paths, and which is connected to no additional vertices in the graph.

First we'll import the Neo4j driver and Pandas libraries:


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

Next let's create an instance of the Neo4j driver which we'll use to execute our queries.


In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

Now let's create a sample graph that we'll run the algorithm against.


In [3]:
create_graph_query = '''
CREATE (nAlice:User {id:'Alice'})
,(nBridget:User {id:'Bridget'})
,(nCharles:User {id:'Charles'})
,(nDoug:User {id:'Doug'})
,(nMark:User {id:'Mark'})
,(nMichael:User {id:'Michael'})
CREATE (nAlice)-[:FRIEND{weight:0.5}]->(nBridget)
,(nAlice)-[:FRIEND{weight:4}]->(nCharles)
,(nMark)-[:FRIEND{weight:1}]->(nDoug)
,(nMark)-[:FRIEND{weight:2}]->(nMichael);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 6, 'relationships-created': 4, 'nodes-created': 6, 'properties-set': 10}


Finally we can run the algorithm by executing the following query:


In [4]:
streaming_query = """
CALL algo.unionFind.stream('User', 'FRIEND', {}) 
YIELD nodeId,setId
RETURN nodeId,setId
LIMIT 20;
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,nodeId,setId
0,73,0
1,151,0
2,163,0
3,164,4
4,211,4
5,212,4
